# RoBERTa-based Manipulation Detection (Binary Classification) ith PEFT/LORA added
This notebook uses `roberta-base` to classify dialogue as manipulative or not using the MentalManip dataset.

In [1]:
!pip install -q transformers
!pip install -q datasets
!pip install -q evaluate
## transformers upgrade
!pip install -q --upgrade transformers

## Datasets need upgrading to work
!pip install -q --upgrade datasets

## LoRA addon
!pip install -q peft accelerate


## Early stopping
from transformers import EarlyStoppingCallback


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.

In [2]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import classification_report



In [3]:
model_ckpt = "roberta-base"

In [4]:
# Load the MentalManip dataset (binary classification)
# Load dataset
dataset = load_dataset("audreyeleven/MentalManip", name="mentalmanip_maj")

print(dataset)


README.md: 0.00B [00:00, ?B/s]

Some datasets params were ignored: ['license']. Make sure to use only valid params for the dataset builder and to have a up-to-date version of the `datasets` library.


mentalmanip_maj.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/4000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'manipulative', 'technique', 'vulnerability'],
        num_rows: 4000
    })
})


In [5]:
# Ensure the 'manipulative' column is class-labeled
dataset = dataset.class_encode_column("manipulative")




Stringifying the column:   0%|          | 0/4000 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

def tokenize_fn(example):
    return tokenizer(example["dialogue"], truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(tokenize_fn, batched=True)
print(tokenized)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'manipulative', 'technique', 'vulnerability', 'input_ids', 'attention_mask'],
        num_rows: 4000
    })
})


In [7]:
## Setup the PEFT addon

from peft import get_peft_model, LoraConfig, TaskType

# Define your LoRA configuration
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,           # Sequence classification
    inference_mode=False,                 # True = inference only
    r=8,                                  # Low-rank dimension
    lora_alpha=16,                        # Scaling factor
    lora_dropout=0.1,                     # Dropout in LoRA layers
    bias="none",
    target_modules=["query", "value"]     # Apply only to LoRA layers
)

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [8]:

base_model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=2)

# Wrap with LoRA
model = get_peft_model(base_model, peft_config)

# Pring trainable layers
for name, module in model.named_modules():
    if "attention" in name and any(k in name for k in ["query", "key", "value"]):
        print(name)


# Optional: Show how many parameters are trainable
model.print_trainable_parameters()





model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


base_model.model.roberta.encoder.layer.0.attention.self.query
base_model.model.roberta.encoder.layer.0.attention.self.query.base_layer
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_dropout
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_dropout.default
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_A
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_A.default
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_B
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_B.default
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_embedding_A
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_embedding_B
base_model.model.roberta.encoder.layer.0.attention.self.query.lora_magnitude_vector
base_model.model.roberta.encoder.layer.0.attention.self.key
base_model.model.roberta.encoder.layer.0.attention.self.value
base_model.model.roberta.encoder.layer.0.attention.

In [9]:
# Split the dataset into training and testing sets
train_test_split = tokenized["train"].train_test_split(test_size=0.2) # Adjust the test_size as needed

# Update the tokenized dataset with the new splits
tokenized["train"] = train_test_split["train"]
tokenized["test"] = train_test_split["test"]

# Make the data work with the nomenclature
tokenized = tokenized.rename_column("manipulative", "labels")

print(tokenized)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'labels', 'technique', 'vulnerability', 'input_ids', 'attention_mask'],
        num_rows: 3200
    })
    test: Dataset({
        features: ['id', 'dialogue', 'labels', 'technique', 'vulnerability', 'input_ids', 'attention_mask'],
        num_rows: 800
    })
})


In [10]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=2  # Binary classification
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
## Stop epochs if overfitting
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,  # Stop after 2 epochs without improvement
    early_stopping_threshold=0.0
)


In [13]:
## Training args

training_args = TrainingArguments(
    output_dir="./bert-binary-manip",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=8,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    run_name="bert-binary-manip",
    report_to="none",
    learning_rate=2e-5,
)

In [15]:
## Evaluation metrics

import evaluate
import numpy as np

accuracy = evaluate.load('accuracy')
precision = evaluate.load('precision')
recall = evaluate.load('recall')
f1 = evaluate.load('f1')

def compute_metrics(p):
    predictions, labels = p
    predictions_argmax = np.argmax(predictions, axis=1)

    return {
        "accuracy": accuracy.compute(predictions=predictions_argmax, references=labels)["accuracy"],
        "precision": precision.compute(predictions=predictions_argmax, references=labels, average='weighted')["precision"],
        "recall": recall.compute(predictions=predictions_argmax, references=labels, average='weighted')["recall"],
        "f1": f1.compute(predictions=predictions_argmax, references=labels, average='weighted')["f1"],
    }

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[early_stopping], # Pass callbacks here
)
trainer.train()

/tmp/ipython-input-16-3997555682.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.549800,0.641415,0.698750,0.652110,0.698750,0.589128
2,0.590600,0.618999,0.697500,0.486506,0.697500,0.573203
3,0.553400,0.605019,0.707500,0.719751,0.707500,0.604019
4,0.651900,0.614472,0.697500,0.486506,0.697500,0.573203
5,0.624800,0.613250,0.697500,0.486506,0.697500,0.573203


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=1000, training_loss=0.6081773681640625, metrics={'train_runtime': 634.2066, 'train_samples_per_second': 40.365, 'train_steps_per_second': 2.523, 'total_flos': 1052444221440000.0, 'train_loss': 0.6081773681640625, 'epoch': 5.0})

In [17]:

##  Easy eval results...
##  Get the log history from the trainer state
log_history = trainer.state.log_history

# Initialize placeholders
last_train_loss = None
last_eval_loss = None

# Iterate through log history to find the last recorded train and eval loss
for log in reversed(log_history):
    if last_eval_loss is None and "eval_loss" in log:
        last_eval_loss = log["eval_loss"]
    if last_train_loss is None and "loss" in log:
        last_train_loss = log["loss"]
    if last_train_loss is not None and last_eval_loss is not None:
        break

# Calculate overfitting ratio
if last_train_loss is not None and last_eval_loss is not None:
    ratio = last_train_loss / last_eval_loss
    print(f"Training Loss: {last_train_loss:.5f}")
    print(f"Validation Loss: {last_eval_loss:.5f}")
    print(f"Overfitting Ratio: {ratio:.5f}")
    if ratio < 0.6:
        print("Overfitting detected!")
    else:
        print("No significant overfitting.")
else:
    print("Could not find both training and evaluation loss in log history.")


Training Loss: 0.62480
Validation Loss: 0.61325
Overfitting Ratio: 1.01883
No significant overfitting.


In [18]:
# Predict on test set
preds = trainer.predict(tokenized["test"])
y_pred = preds.predictions.argmax(-1)
y_true = preds.label_ids

# Detailed classification report
print(classification_report(y_true, y_pred, target_names=["non-manipulative", "manipulative"]))


                  precision    recall  f1-score   support

non-manipulative       0.75      0.05      0.09       242
    manipulative       0.71      0.99      0.83       558

        accuracy                           0.71       800
       macro avg       0.73      0.52      0.46       800
    weighted avg       0.72      0.71      0.60       800



In [23]:
print(train_test_split["train"])
from collections import Counter
print("Label distribution in training set:", Counter(train_test_split["train"]['manipulative']))
print("Label distribution in validation set:", Counter(train_test_split["test"]['manipulative']))


Dataset({
    features: ['id', 'dialogue', 'manipulative', 'technique', 'vulnerability', 'input_ids', 'attention_mask'],
    num_rows: 3200
})
Label distribution in training set: Counter({1: 2260, 0: 940})
Label distribution in validation set: Counter({1: 558, 0: 242})


In [25]:

preds = trainer.predict(train_test_split["test"])
import numpy as np
predicted_labels = np.argmax(preds.predictions, axis=1)
print("Predicted label distribution:", dict(Counter(predicted_labels)))


Predicted label distribution: {np.int64(1): 784, np.int64(0): 16}


Adjusted learning rate to: 2e-05
